# Práctica 2 Parte 2

## Descripción

La segunda parte consiste en reimplementar nuestro método numérico realizado en la práctica 1 con niveles de BLAS, cómputo en paralelo (CPU/GPU), con compilación a **C** (por ejemplo vía _cython_, _rcpp_) o **julia** guiándose del perfilamiento de memoria, uso de procesador o tiempo de ejecución del paquete.
Para consultar la información de nuestro paquete, vea: [reporte de la práctica 1, segunda parte](https://github.com/optimizacion-2-2022-gh-classroom/practica-1-segunda-parte-LuzVerde23/blob/main/reporte_equipo_2_parte_2_practica_1.ipynb).


In [1]:
import time
import pandas as pd
import networkx as nx
import random
import cython
from MaxFlowAeiu import MaxFlowAeiu

Planeamos correr los mismos ejemplos de la [práctica 2, primera parte](https://github.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/blob/main/reporte_equipo_2_parte_1_practica_2.ipynb), con la nueva implementación, para la cual se puede consultar el desarrollo en la carpeta [notebooks_apoyo](https://github.com/optimizacion-2-2022-gh-classroom/practica-2-segunda-parte-EddOselotl/tree/main/notebooks_apoyo). Y al final comparar los tiempos obtenidos entre las dos versiones de nuestro paquete MaxFlowAeiu.

## Ejemplo 1
Para comenzar usaremos el ejemplo de la práctica uno

In [2]:
url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
d = pd.read_csv(url_d,header=None)
arreglo = d.values.tolist()

In [3]:
%time
start_time = time.time()
MF=MaxFlowAeiu(arreglo)
MF.ford_fulkerson()
print("The maximum flow in this network is {}".format(MF.get_maximumflow()))
end_time = time.time()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
maximun flow is 1480
The maximum flow in this network is 1480


In [4]:
secs_1 = end_time-start_time
print("El algoritmo para el ejemplo 1 tomó",secs_1,"segundos" )

El algoritmo para el ejemplo 1 tomó 0.012638330459594727 segundos


# Ejemplo 2
Seguimos con un ejemplo de menor escala que trata sobre el transporte de personas en una empresa de autobúses

In [5]:
d2 = [[0,188,240,160,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,170,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,210,180,0,0,0,0,0],
                    [0,0,0,0,0,140,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,130,90,0,0],
                    [0,0,0,0,0,0,0,0,130,0,0,0,0],
                    [0,0,0,0,0,0,0,0,150,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,160,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,210],
                    [0,0,0,0,0,0,0,0,0,0,0,0,110],
                    [0,0,0,0,0,0,0,0,0,0,0,0,80],
                    [0,0,0,0,0,0,0,0,0,0,0,0,170],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0]]
arreglo2 = d2

In [6]:
%time
start_time = time.time()
MF2=MaxFlowAeiu(arreglo2)
MF2.ford_fulkerson()
print("The maximum flow in this network is {}".format(MF2.get_maximumflow()))
end_time = time.time()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
maximun flow is 540
The maximum flow in this network is 540


In [7]:
secs_2 = end_time-start_time
print("El algoritmo para el ejemplo 2 tomó",secs_2,"segundos" )

El algoritmo para el ejemplo 2 tomó 0.0005862712860107422 segundos


## Ejemplo 3
Ahora probaremos con una matriz más grande sobre el transporte de pasajeros en aerolíneas en Estados Unidos

In [8]:
d3=pd.read_csv("https://raw.githubusercontent.com/sid-7/Airline_Maximum_Flow/master/flights.csv")
d3=d3.drop(['Unnamed: 0', 'Depature', 'Arrival'], axis=1)
d3=d3.groupby(['Source', 'Destination'], as_index=False)['capacity'].sum()
d3['capacity'] = pd.to_numeric(d3['capacity'], errors='coerce')
new_row = pd.DataFrame([['JFK','JFK', 0], ['LAX','LAX', 0]],
                   columns=['Source', 'Destination', 'capacity'])
d3 = pd.concat([d3, new_row])
d3 = d3.pivot(index="Source", columns="Destination", values="capacity").fillna(0)
column_to_move1 = d3.pop("LAX")
column_to_move2 = d3.pop("JFK")
d3.insert(0, "LAX", column_to_move1)
d3.insert(9, "JFK", column_to_move2)
t1 = ['JFK','LAX']
t2 = ['JFK']
t3 = ['LAX']

a = d3.loc[[i for i in d3.index if i not in t1], :]
b = d3.loc[t2, :]
c = d3.loc[t3, :]

d3=pd.concat([c, a, b])
arreglo3 = d3.values.tolist()

In [9]:
%time
start_time = time.time()
MF3=MaxFlowAeiu(arreglo3)
MF3.ford_fulkerson()
print("The maximum flow in this network is {}".format(MF3.get_maximumflow()))
end_time = time.time()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs
maximun flow is 6427
The maximum flow in this network is 6427


In [10]:
secs_3 = end_time-start_time
print("El algoritmo para el ejemplo 3 tomó",secs_3,"segundos" )

El algoritmo para el ejemplo 3 tomó 0.0005733966827392578 segundos


## Ejemplo 4
Ahora usaremos una matriz mucho más grande

In [16]:
# Especificar el tamaño de la red n = número de nodos, m = número de ramas
n = 1000
m = 5000
network = nx.gnm_random_graph(n, m, directed=True)
nodos = list(network.nodes)
ramas = list(network.edges)
# Crear arreglos de: nodo de envío, nodo de recepción y capacidad del enlace
env = []
rec = []
cap = []
for i in range(len(ramas)):
    env.append(ramas[i][0])
    rec.append(ramas[i][1])
    cap.append(random.randint(20,100)) # La capacidad del enlace se determinade forma aleatoria con valores entre 20 y 100

In [17]:
# Se crea el data frame o matriz de incidencias de los nodos. Se asegura que el nodo fuente sea el 0 y solo tenga ramas saliendo de él y que el último nodo solo tenga ramas entrando (sumidero)
d4 = pd.DataFrame(0, index=list(range(0,len(nodos))), columns=list(range(0,len(nodos))))
aux = 0
for n in env:
    if rec[aux] == 0:
        d4.iloc[rec[aux],n] = cap[aux]
    elif n == max(nodos):
        d4.iloc[rec[aux],n] = cap[aux]
    else:
        d4.iloc[n,rec[aux]] = cap[aux]
    aux += 1

In [18]:
arreglo4 = d4.values.tolist()

In [19]:
%time
start_time = time.time()
MF4=MaxFlowAeiu(arreglo4)
MF4.ford_fulkerson()
print("The maximum flow in this network is {}".format(MF4.get_maximumflow()))
end_time = time.time()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
maximun flow is 393
The maximum flow in this network is 393


In [20]:
secs_4 = end_time-start_time
print("El algoritmo para el ejemplo 4 tomó",secs_4,"segundos" )

El algoritmo para el ejemplo 4 tomó 67.02914547920227 segundos


## Reporte de resultados

La realización de estos 4 ejercicios se hizo en un ambiente virtual (instancia de AWS) con las herramientas de Kale, Minikube y Kubeflow. En la siguiente imagen se muestra el pipeline ejecutado con los cuatro ejemplos corriendo de manera satisfactoria:

<p align = "center">
    <img src="images/kale_flow.png" width="1329" height="911" />

Adicionalmente se realizó la misma ejecución de los 4 ejercicios, pero en el ambiente virtual generado para este repositorio desde la funcionalidad de _Binder_. En la siguiente tabla se muestran los resultados obtenidos en ambas ejecuciones:

In [16]:
# Cambiar esta tabla
results = {
    'Experimento' : [1, 1, 2, 2, 3, 3, 4, 4],
    'Tamaño_Red': ["Mediana", "Mediana", "Pequeña", "Pequeña", "Pequeña", "Pequeña", "Grande", "Grande" ],
    'Nodos': [44, 44, 13, 13, 10, 10, 1000, 1000],
    'Ramas': [61, 61, 16, 16, 72, 72, 5000, 5000],
    'Flujo_Max': [1480, 1480, 540, 540, 6427, 6427, 643, 513],
    'Exitoso': ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si'],
    'CPU_Time_(ms)': [4.65, 7e-3, 3.37, 6e-3, 7.51, 7e-3, 26e3, 6e-3],
    'Wall_Time_(ms)': [7.68, 14.1e-3, 3.15, 11.9e-3, 5.62, 12.9e-3, 26e3, 12.2e-3],
    'Fecha': ["22-04-2022", "23-04-2022", "22-04-2022", "23-04-2022", "22-04-2022", "23-04-2022", "22-04-2022", "23-04-2022"],
    'Instancia': ["AWS", "Binder", "AWS", "Binder", "AWS", "Binder", "AWS", "Binder"]
}
r = pd.DataFrame(results)
r

,Experimento,Tamaño_Red,Nodos,Ramas,Flujo_Max,Exitoso,CPU_Time_(ms),Wall_Time_(ms),Fecha,Instancia
0,1,Mediana,44,61,1480,Si,4.650,7.6800,22-04-2022,AWS
1,1,Mediana,44,61,1480,Si,0.007,0.0141,23-04-2022,Binder
2,2,Pequeña,13,16,540,Si,3.370,3.1500,22-04-2022,AWS
3,2,Pequeña,13,16,540,Si,0.006,0.0119,23-04-2022,Binder
4,3,Pequeña,10,72,6427,Si,7.510,5.6200,22-04-2022,AWS
5,3,Pequeña,10,72,6427,Si,0.007,0.0129,23-04-2022,Binder
6,4,Grande,1000,5000,643,Si,26000.000,26000.0000,22-04-2022,AWS
7,4,Grande,1000,5000,513,Si,0.006,0.0122,23-04-2022,Binder


Así mismo, comparamos los 4 ejemplos con la tabla de la práctica anterior para comparar los tiempos:

In [21]:
# Tabla anterior, no cambiar
results = {
    'Experimento' : [1, 1, 2, 2, 3, 3, 4, 4],
    'Tamaño_Red': ["Mediana", "Mediana", "Pequeña", "Pequeña", "Pequeña", "Pequeña", "Grande", "Grande" ],
    'Nodos': [44, 44, 13, 13, 10, 10, 1000, 1000],
    'Ramas': [61, 61, 16, 16, 72, 72, 5000, 5000],
    'Flujo_Max': [1480, 1480, 540, 540, 6427, 6427, 643, 513],
    'Exitoso': ['Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si'],
    'CPU_Time_(ms)': [2e-3, 11e-3, 2e-3, 11e-3, 3e-3, 11e-3, 3e-3, 11e-3],
    'Wall_Time_(ms)': [4.77e-3, 21.7e-3, 4.53e-3, 21.2e-3, 4.77e-3, 21.5e-3, 4.77e-3, 22.2e-3],
    'Fecha': ["22-05-2022", "23-05-2022", "22-05-2022", "23-05-2022", "22-05-2022", "23-05-2022", "22-05-2022", "23-05-2022"],
    'Instancia': ["AWS", "Binder", "AWS", "Binder", "AWS", "Binder", "AWS", "Binder"]
}
r = pd.DataFrame(results)
r

,Experimento,Tamaño_Red,Nodos,Ramas,Flujo_Max,Exitoso,CPU_Time_(ms),Wall_Time_(ms),Fecha,Instancia
0,1,Mediana,44,61,1480,Si,0.002,0.00477,22-05-2022,AWS
1,1,Mediana,44,61,1480,Si,0.000,0.00000,23-05-2022,Binder
2,2,Pequeña,13,16,540,Si,0.002,0.00453,22-05-2022,AWS
3,2,Pequeña,13,16,540,Si,0.000,0.00000,23-05-2022,Binder
4,3,Pequeña,10,72,6427,Si,0.003,0.00477,22-05-2022,AWS
5,3,Pequeña,10,72,6427,Si,0.000,0.00000,23-05-2022,Binder
6,4,Grande,1000,5000,643,Si,0.003,0.00477,22-05-2022,AWS
7,4,Grande,1000,5000,513,Si,0.000,0.00000,23-05-2022,Binder


### Características de las instancias

---

**BINDER**

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              72
On-line CPU(s) list: 0-71
Thread(s) per core:  2
Core(s) per socket:  18
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz
Stepping:            4
CPU MHz:             1000.015
CPU max MHz:         2300.0000
CPU min MHz:         1000.0000
BogoMIPS:            4600.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            25344K
```
---

**AWS**



## Conclusiones

Podemos observar una clara mejoría en la ejecución de todos nuestros ejemplos gracias a la implementación que hicimos usando Cython, en el experimento ejecutado en la instancia de AWS con una máquina `tipo de máquina` vemos que el desempeño de nuestro algoritmo da muy buenos resultados en cuanto a tiempo de ejecución. 



## Referencias
* [1] [Palacios E. (2022) Libro de Optimización](https://itam-ds.github.io/analisis-numerico-computo-cientifico/4.optimizacion_en_redes_y_prog_lineal/4.2/Definiciones_generales_de_flujo_en_redes.html)
* [2] [Dumora c. el all. Data Oriented Algorithm for Real Time Estimation of Flow Rates and Flow Directions in Water Distribution Network](https://arxiv.org/pdf/1807.10147.pdf)
* [3] [Max Flow Problem Introduction](https://www.geeksforgeeks.org/max-flow-problem-introduction/)
* [4] [Ford-Fulkerson Algorithm](https://www.programiz.com/dsa/ford-fulkerson-algorithm)
* [5] [Algoritmo de Ford-Fulkerson - Ford–Fulkerson algorithm](https://upwikies.top/wiki/Ford%e2%80%93Fulkerson_algorithm)
* [6] [Oviedo J. (2008) Algoritmo de Ford-Fulkerson Mejorado](http://www.ptolomeo.unam.mx:8080/jspui/bitstream/132.248.52.100/2387/1/gonzalezoviedo.pdf)
* [7] [Building a Smarter (and Cheaper) School Bus System: How a Boston-MIT Partnership Led to New Routes That Are 20% More Efficient and Saved the District $5 Million](https://www.the74million.org/article/building-a-smarter-and-cheaper-school-bus-system-how-a-boston-mit-partnership-led-to-new-routes-that-are-20-more-efficient-use-400-fewer-buses-save-5-million/)
* [8] [Optimazation examples](https://vitalflux.com/convex-optimization-explained-concepts-examples/)
* [9] [Breadth First Search or BFS for a Graph](https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/)
* [10] [Create your own Python package and publish it into PyPI](https://towardsdatascience.com/create-your-own-python-package-and-publish-it-into-pypi-9306a29bc116)
* [11] [Speeding up Python code with Cython](https://smirnov-am.github.io/cython/)
* [12] [Cython’s Documentation](http://docs.cython.org/en/latest/index.html)